In [19]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests
import plotly.express as px
from dotenv import load_dotenv
from pathlib import Path

In [20]:
load_dotenv()

# Получаем даты из переменных окружения
start_date = os.getenv("DATE_BEGIN")
end_date = os.getenv("DATE_END")
api_url = os.getenv("API_URL")

# Делаем две ссылки
visit_url = f"{api_url}/visits?begin={start_date}&end={end_date}"
reg_url = f"{api_url}/registrations?begin={start_date}&end={end_date}"

# Тянем данные
visits = requests.get(visit_url).json()
registrations = requests.get(reg_url).json()
df_registrations = pd.DataFrame(registrations)
df_visits = pd.DataFrame(visits)

In [21]:
def summary_fun():
    summary_regs = df_registrations.describe()
    summary_visits = df_visits.describe()
    return summary_regs, summary_visits

In [22]:
import pandas as pd
import requests
import numpy as np

def conversion_fun():
    # Получаем данные визитов и регистраций
    visits = requests.get(visit_url).json()
    registrations = requests.get(reg_url).json()

    # Конвертируем в датафрейм
    df_visits = pd.DataFrame(visits)
    df_registrations = pd.DataFrame(registrations)

    # Убираем ботов
    filtered_visits = df_visits[~df_visits["user_agent"].str.contains("bot", case=False, na=False)]
    filtered_visits["datetime"] = pd.to_datetime(filtered_visits["datetime"])
    df_registrations["datetime"] = pd.to_datetime(df_registrations["datetime"])

    # Сортируем и берем последний визит
    last_visit = filtered_visits.sort_values(by=['visit_id', 'datetime']).drop_duplicates(subset=['visit_id'], keep='last')
    last_visit["date_group"] = last_visit["datetime"].dt.to_period('D').dt.start_time.dt.date
    df_visit = last_visit.groupby(["platform", "date_group"]).size().reset_index(name="visits")

    # Регистрации по датам и платформам
    df_registrations["date_group"] = df_registrations["datetime"].dt.date  # Исправлено: используем df_registrations
    df_reg = df_registrations.groupby(["platform", "date_group"]).size().reset_index(name="registrations")

    # Объединяем и вычисляем конверсию
    df_conversion = pd.merge(df_visit, df_reg, on=["platform", "date_group"], how='outer').fillna(0)  # Добавлено: обработка NaN
    df_conversion["conversion"] = 100 * df_conversion["registrations"] / df_conversion["visits"].replace(0, np.nan)
    df_conversion = df_conversion[["date_group", "platform", "visits", "registrations", "conversion"]]
    df_conversion = df_conversion.sort_values(by=["date_group", "platform"]).reset_index(drop=True)

    # Сохраняем в JSON
    df_conversion.to_json("./conversion.json", orient='records')
    return df_conversion

In [23]:
def process_ad_data(df_conversion):
    df_company = pd.read_csv("ads.csv")
    df_company["date"] = pd.to_datetime(df_company["date"])
    df_company["date_group"] = df_company["date"].dt.date
    df_company.drop_duplicates(inplace=True)

    start_date = pd.to_datetime(os.getenv("DATE_BEGIN")).date()
    end_date = pd.to_datetime(os.getenv("DATE_END")).date()
    df_company = df_company[(df_company["date_group"] >= start_date) & (df_company["date_group"] <= end_date)]
    df_company = df_company.groupby(["date_group", "utm_campaign"], as_index=False)["cost"].sum()

    df_conversion["date_group"] = pd.to_datetime(df_conversion["date_group"]).dt.date
    ads = pd.merge(df_conversion, df_company, on=["date_group"], how="left").fillna("none")
    ads = ads[["date_group", "visits", "registrations", "cost", "utm_campaign"]]
    ads = ads.reset_index(drop=True).sort_values(by=["date_group"])

    ads.to_json("./ads.json")
    return ads

In [ ]:
# Вызовы функций
df_conversion = conversion_fun()
ads = process_ad_data(df_conversion)

In [ ]:
# Готовим данные для conversion
conversion = df_conversion.copy()
conversion["date_group"] = pd.to_datetime(conversion["date_group"])
conversion["month"] = conversion["date_group"].dt.to_period("M")

# Готовим данные для ads
ads["date_group"] = pd.to_datetime(ads["date_group"])
ads["month"] = ads["date_group"].dt.to_period("M")

# Создаем сводную таблицу
pivot_conversion = pd.pivot_table(conversion, index="month", columns="platform", values=["visits", "registrations", "conversion"], aggfunc={"visits": "sum", "registrations": "sum", "conversion": "mean"}, fill_value=0)
pivot_ads = pd.pivot_table(ads, index="month", columns="utm_campaign", values=["visits", "registrations", "cost"], aggfunc="sum", fill_value=0)

# Подсчитываем суммарные визиты и регистрации по месяцам
total_visits = pivot_conversion["visits"].sum(axis=1)
total_regs = pivot_conversion["registrations"].sum(axis=1)
total_convers = pivot_conversion["conversion"]

In [ ]:
p = Path.cwd()
d = p / 'charts'
d.mkdir(exist_ok=True)

In [ ]:
def plot_total_visits(total_visits):
    plt.figure(figsize=(12, 6))
    bars = total_visits.plot(kind="bar", color="skyblue")
    for bar in bars.patches:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval, int(yval), va="bottom", ha="center")
    plt.title("Суммарные визиты по месяцам", fontsize=16)
    plt.xlabel("Месяц", fontsize=12)
    plt.ylabel("Количество визитов", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis="y")
    plt.tight_layout()
    plt.savefig("./charts/total_visits_per_month.png")
    plt.show()

plot_total_visits(total_visits)

In [ ]:
def plot_visits_by_platform():
    ax = pivot_conversion["visits"].plot(kind="bar", stacked=True, figsize=(14, 7))
    plt.title("Visits by Platform (Stacked)")
    plt.xlabel("Date Group (Month)")
    plt.ylabel("Number of Visits")
    plt.xticks(rotation=45)
    plt.legend(title="Platform")
    plt.tight_layout()
    for i in range(pivot_conversion["visits"].shape[0]):
        cumulative_height = 0
        for j in range(pivot_conversion["visits"].shape[1]):
            value = pivot_conversion["visits"].iloc[i, j]
            cumulative_height += value
            if value > 0:
                ax.text(
                    i,
                    cumulative_height - value / 2,
                    str(value),
                    ha="center",
                    va="center",
                )
    plt.savefig("./charts/visits_by_platform.png")
    plt.show()
plot_visits_by_platform()

In [ ]:
def plot_total_regs():
    plt.figure(figsize=(12, 6))
    bars = total_regs.plot(kind="bar", color="skyblue")
    for bar in bars.patches:
        yval = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2, yval, int(yval), va="bottom", ha="center"
        )
    plt.title("Суммарные регистрации по месяцам", fontsize=16)
    plt.xlabel("Месяц", fontsize=12)
    plt.ylabel("Количество регистраций", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis="y")
    plt.tight_layout()
    plt.savefig("./charts/total_regs_per_month.png")
    plt.show()
plot_total_regs()

In [ ]:
def plot_conversion_by_platform():
    plt.figure(figsize=(14, 7))
    for platform in pivot_conversion["conversion"].columns:
        plt.plot(
            total_convers.index.astype(str),
            total_convers[platform],
            marker="o",
            linestyle="-",
            label=f"Conversion {platform}",
        )
        for i, val in enumerate(total_convers[platform]):
            plt.annotate(
                f"{val:.2f}%",
                (
                    total_convers.index[i].strftime("%Y-%m"),
                    total_convers[platform].iloc[i],
                ),
                textcoords="offset points",
                xytext=(0, 10),
                ha="center",
            )
    plt.title("Конверсии по платформам", fontsize=16)
    plt.xlabel("Месяц", fontsize=14)
    plt.ylabel("Конверсия (%)", fontsize=14)
    plt.xticks(rotation=45)
    plt.legend(title="Платформа")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("./charts/conversion_by_platform.png")
    plt.show()
plot_conversion_by_platform()

In [ ]:
def plot_ads_visits():
    plt.figure(figsize=(14, 7))
    ax = pivot_ads["visits"].plot(kind="bar", stacked=True)
    plt.title("Visits by UTM Campaign (Stacked)")
    plt.xlabel("Date Group (Month)")
    plt.ylabel("Number of Visits")
    plt.xticks(rotation=45)
    plt.legend(title="UTM Campaign")
    plt.tight_layout()
    for i in range(pivot_ads["visits"].shape[0]):
        cumulative_height = 0
        for j in range(pivot_ads["visits"].shape[1]):
            value = pivot_ads["visits"].iloc[i, j]
            cumulative_height += value
            if value > 0:
                ax.text(
                    i,
                    cumulative_height - value / 2,
                    str(value),
                    ha="center",
                    va="center",
                )
    plt.savefig("./charts/ads_visits_by_campaign.png")
    plt.show()
plot_ads_visits()

In [ ]:
def plot_ads_regs():
    plt.figure(figsize=(14, 7))
    ax = pivot_ads["registrations"].plot(kind="bar", stacked=True)
    plt.title("Registrations by UTM Campaign (Stacked)")
    plt.xlabel("Date Group (Month)")
    plt.ylabel("Number of Registrations")
    plt.xticks(rotation=45)
    plt.legend(title="UTM Campaign")
    plt.tight_layout()
    for i in range(pivot_ads["registrations"].shape[0]):
        cumulative_height = 0
        for j in range(pivot_ads["registrations"].shape[1]):
            value = pivot_ads["registrations"].iloc[i, j]
            cumulative_height += value
            if value > 0:
                ax.text(
                    i,
                    cumulative_height - value / 2,
                    str(value),
                    ha="center",
                    va="center",
                )
    plt.savefig("./charts/ads_regs_by_campaign.png")
    plt.show()
plot_ads_regs()

In [ ]:
def plot_ads_cost():
    plt.figure(figsize=(14, 7))
    ax = pivot_ads["cost"].plot(kind="bar", stacked=True)
    plt.title("Cost by UTM Campaign (Stacked)")
    plt.xlabel("Date Group (Month)")
    plt.ylabel("Total Cost")
    plt.xticks(rotation=45)
    plt.legend(title="UTM Campaign")
    plt.tight_layout()
    
    for i in range(pivot_ads["cost"].shape[0]):
        cumulative_height = 0
        for j in range(pivot_ads["cost"].shape[1]):
            # Получаем значение и обрабатываем потенциальные нечисловые значения
            try:
                # Пытаемся конвертировать в float, если возможно
                value = float(pivot_ads["cost"].iloc[i, j])
                cumulative_height += value
                if value > 0:
                    ax.text(
                        i,
                        cumulative_height - value / 2,
                        f"{value:.2f}",  # Форматируем строку с 2 десятичными знаками
                        ha="center",
                        va="center",
                    )
            except (ValueError, TypeError):
                # Пропускаем это значение, если его не удается конвертировать в float
                continue
    
    plt.savefig("./charts/ads_cost_by_campaign.png")
    plt.show()

plot_ads_cost()

In [ ]:
def plot_conversion_stats(month="2023-07"):
    global conversion  # Ссылаемся на глобальную переменную
    # Фильтруем данные по выбранному месяцу
    filtered_conversion = conversion[conversion["month"] == month]

    # Создаем сводную таблицу
    daily_conversion = pd.pivot_table(
        filtered_conversion,
        index="date_group",
        columns="platform",
        values=["visits", "registrations", "conversion"],
        aggfunc={"visits": "sum", "registrations": "sum", "conversion": "mean"},
        fill_value=0,
    )

    # Получаем отдельные данные для графиков
    visits = daily_conversion["visits"]
    registrations = daily_conversion["registrations"]
    conversion_data = daily_conversion["conversion"]  # Переименуем переменную

    # Настраиваем графики
    plt.figure(figsize=(15, 10))

    # График визитов
    plt.subplot(3, 1, 1)
    for platform in visits.columns:
        plt.plot(visits.index, visits[platform], label=platform)
        for x, y in zip(visits.index, visits[platform]):
            plt.text(x, y, str(y), fontsize=8, ha="right")  # Подписи на графике
    plt.title("Визиты по платформам")
    plt.ylabel("Количество визитов")
    plt.legend()
    plt.grid()

    # График регистраций
    plt.subplot(3, 1, 2)
    for platform in registrations.columns:
        plt.plot(registrations.index, registrations[platform], label=platform)
        for x, y in zip(registrations.index, registrations[platform]):
            plt.text(x, y, str(y), fontsize=8, ha="right")  # Подписи на графике
    plt.title("Регистрации по платформам")
    plt.ylabel("Количество регистраций")
    plt.legend()
    plt.grid()

    # График конверсий
    plt.subplot(3, 1, 3)
    for platform in conversion_data.columns:  # Используем новое имя переменной
        plt.plot(conversion_data.index, conversion_data[platform], label=platform)
        for x, y in zip(conversion_data.index, conversion_data[platform]):
            plt.text(x, y, f"{y:.2f}", fontsize=8, ha="right")  # Подписи на графике
    plt.title("Конверсии по платформам")
    plt.ylabel("Средняя конверсия")
    plt.legend()
    plt.grid()

    plt.tight_layout()

    # Сохранение графика
    plt.savefig("./charts/conversion_stats_" + month.replace("-", "") + ".png")

    # Показ графика
    plt.show()
plot_conversion_stats("2023-07")